In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.tsa.api as smt

In [6]:
df = pd.read_csv('../data/Winter negative temperature sum in deg C 1781  1988.csv', index_col=0)
df['t'] = pd.to_datetime({'year': df.index.values + 1780,
                          'month': 1,
                          'day': 1})
df.head()

,x,t
1,272.3,1782-01-01
2,207.8,1783-01-01
3,545.7,1784-01-01
4,557.5,1785-01-01
5,260.6,1786-01-01


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 208 entries, 1 to 208
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   x       208 non-null    float64       
 1   t       207 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 4.9 KB
